In [1]:
import requests
import pandas as pd
import time

# Konfigurasi API ThingSpeak
channel_id = 2326256
read_api_key = 'BKIJNOYDM205J2HG'
url = f'https://api.thingspeak.com/channels/{channel_id}/feeds.json'

# Dapatkan jumlah total data feeds (last_entry_id) dari API ThingSpeak
def get_last_entry_id():
    response = requests.get(f'https://api.thingspeak.com/channels/{channel_id}/feeds.json', params={'api_key': read_api_key, 'results': 1})
    if response.status_code == 200:
        data = response.json()
        return data['channel']['last_entry_id']
    else:
        print(f"Failed to fetch last_entry_id. HTTP Status code: {response.status_code}")
        return None

# Jumlah maksimum data per permintaan (maksimal 8000 menurut dokumentasi ThingSpeak)
max_results_per_page = 8000

# Inisialisasi variabel untuk menampung semua data
all_data = []

# Ambil nilai last_entry_id dari channel
last_entry_id = get_last_entry_id()
if last_entry_id is None:
    print("Could not fetch last_entry_id. Exiting...")
    exit()

# Parameter paginasi
page = 1
total_results = 0

# Loop untuk mengambil data dari semua halaman
while total_results < last_entry_id:
    # Parameter untuk permintaan API
    params = {
        'api_key': read_api_key,
        'results': max_results_per_page,
        'offset': (page - 1) * max_results_per_page
    }

    # Mengirim permintaan GET ke ThingSpeak
    response = requests.get(url, params=params)

    # Cek apakah permintaan berhasil
    if response.status_code == 200:
        # Mengambil data feed dalam format JSON
        data = response.json()
        feeds = data['feeds']

        # Jika tidak ada data lagi, keluar dari loop
        if not feeds:
            break

        # Tambahkan data feed ke list all_data
        all_data.extend(feeds)

        # Tampilkan jumlah data yang sudah diambil dan iterasi ke berapa
        total_results += len(feeds)
        print(f"Iteration {page}: Fetched {total_results} total records so far...")

        # Tunggu sejenak agar tidak membanjiri server ThingSpeak
        time.sleep(1)

        # Lanjutkan ke halaman berikutnya
        page += 1
    else:
        print(f"Failed to fetch data on page {page}. HTTP Status code: {response.status_code}")
        break

# Membuat DataFrame Pandas dari semua data yang dikumpulkan
df = pd.DataFrame(all_data)

# Pilih kolom-kolom yang ingin disimpan, misalnya waktu dan field1 hingga field8
df = df[['created_at', 'field1', 'field2', 'field3', 'field4', 'field8']]

# Ganti nama kolom untuk lebih deskriptif
df.columns = ['Time', 'Suhu', 'Kelembapan', 'Tekanan', 'Embun', 'Volt']

# Simpan DataFrame ke file CSV
df.to_csv('Cuaca_All_Data.csv', index=False)

# Tampilkan DataFrame untuk pengecekan
df.tail(10)


Iteration 1: Fetched 8000 total records so far...
Iteration 2: Fetched 16000 total records so far...
Iteration 3: Fetched 24000 total records so far...
Iteration 4: Fetched 32000 total records so far...
Iteration 5: Fetched 40000 total records so far...
Iteration 6: Fetched 48000 total records so far...


KeyboardInterrupt: 

In [6]:
datahari = (lambda: _deepnote_execute_sql('SELECT *\nFROM \'Cuaca_All_Data.csv\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT *\nFROM \'Cuaca_All_Data.csv\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
datahari

,Time,Suhu,Kelembapan,Tekanan,Embun,Volt
0,2024-09-29 18:50:52,26.06,92.61,1010.92,24.77,4.04
1,2024-09-29 18:51:52,26.05,92.60,1010.89,24.76,4.04
2,2024-09-29 18:52:52,26.05,92.63,1010.91,24.76,4.04
3,2024-09-29 18:53:52,26.05,92.62,1010.86,24.76,4.04
4,2024-09-29 18:54:52,26.04,92.59,1010.82,24.74,4.04
...,...,...,...,...,...,...
463995,2024-10-05 11:17:22,28.12,86.19,1011.11,25.59,4.05
463996,2024-10-05 11:18:22,28.10,86.41,1011.07,25.62,4.05
463997,2024-10-05 11:19:22,28.11,86.51,1011.07,25.65,4.05
463998,2024-10-05 11:20:22,28.09,86.36,1011.12,25.60,4.05


In [7]:
# Mencari 10 nilai suhu terendah beserta tanggalnya
lowest_temperatures = datahari.nsmallest(10, 'Suhu')[['Time', 'Suhu']]
# Mencari 10 nilai suhu tertinggi beserta tanggalnya
highest_temperatures = datahari.nlargest(10, 'Suhu')[['Time', 'Suhu']]

# Menampilkan hasil
print(lowest_temperatures)
# Menampilkan hasil
print(highest_temperatures)

                    Time  Suhu
214  2024-09-29 22:28:55  25.0
217  2024-09-29 22:31:55  25.0
219  2024-09-29 22:33:55  25.0
223  2024-09-29 22:37:55  25.0
228  2024-09-29 22:42:55  25.0
8214 2024-09-29 22:28:55  25.0
8217 2024-09-29 22:31:55  25.0
8219 2024-09-29 22:33:55  25.0
8223 2024-09-29 22:37:55  25.0
8228 2024-09-29 22:42:55  25.0
                     Time   Suhu
3498  2024-10-02 05:29:31  34.26
11498 2024-10-02 05:29:31  34.26
19498 2024-10-02 05:29:31  34.26
27498 2024-10-02 05:29:31  34.26
35498 2024-10-02 05:29:31  34.26
43498 2024-10-02 05:29:31  34.26
51498 2024-10-02 05:29:31  34.26
59498 2024-10-02 05:29:31  34.26
67497 2024-10-02 05:29:31  34.26
75497 2024-10-02 05:29:31  34.26


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>